# 准备数据

In [1]:
import sys

sys.path.append('../')

from alignment_utils import one_file_per_process
from alignment_utils import extract_and_combine_numbers
from alignment_utils import longest_increasing_subsequence_index
from alignment_utils import find_answer_split_str
from alignment_utils import find_next_question_index
from alignment_utils import refine_answers
from alignment_utils import match_specific_from_end
from alignment_utils import answer_area_str_process
from alignment_utils import generate_answer_area_string
from alignment_utils import align_answers_in_questions
from alignment_utils import match_specific_from_start_by_lines

In [2]:
import os
import glob
from pathlib import Path

examination_paper_list = []

path = Path("./answer_markdown")

for file in path.glob("*.md"):
    with open(file, "r", encoding="utf-8") as f:
        examination_paper_list.append({
            "file_path": str(file),
            "text": one_file_per_process(f.read())
        })
len(examination_paper_list)

1615

In [3]:
import re

def type_of_judgment(text):
    # 拆分成行
    lines = text.splitlines()
    question_number_list = []
    
    pattern = r'^[一二三四五六七八九十][、．.]'
    for i in range(len(lines)):
        line = lines[i]
        match = re.match(pattern, line)
        if match:
#             print(match.group(0)[0])
            if i+1 < len(lines):
                next_line = lines[i+1]
                question_number_list.append(extract_and_combine_numbers(next_line))
    
    # 计算1在数组中出现的次数
    count_of_ones = question_number_list.count(1)
    
    return count_of_ones > len(question_number_list) / 2


# 寻找题目

In [4]:
def split_question(question, topic_number=0, target_topic_number=0):
    topic_number = topic_number if topic_number else extract_and_combine_numbers(question) # 1 
    recursion = False if target_topic_number else True # False
    target_topic_number = target_topic_number if target_topic_number else topic_number + 1 # 3

    split_question_list = []
    search_question = question
    search_number = topic_number+1
    
    while True:
        current_question, next_question= match_specific_from_start_by_lines(search_question, search_number) 
        
        if current_question is not None:
            split_question_list.append(current_question)
            search_question = next_question
        else:
            break
            
        if target_topic_number - search_number <= 0:

            if not recursion:
                break       
        search_number = search_number+1
        
    split_question_list.append(search_question)
   
    return split_question_list

In [5]:
def find_continuous_sequence(all_question):
    new_all_question = []
    for index, question in enumerate(all_question):
        if index+1 == len(all_question):
            new_all_question+=split_question(question)
            break
        topic_number = extract_and_combine_numbers(question)
        next_topic_number = extract_and_combine_numbers(all_question[index+1])
        if topic_number+1 == next_topic_number:
            new_all_question.append(question)
            continue
        new_all_question += split_question(question,topic_number,next_topic_number-1)
        
    return new_all_question

In [6]:
def get_all_question(text):
    # 拆分成行
    lines = text.splitlines()
    
    # 定义不准确的题目列表    
    inaccuracy_question = []

    # 从0的位置寻找第一道题
    index = find_next_question_index(0, lines)
    
    while index < len(lines):
        # 寻找下一个题目的index
        next_index = find_next_question_index(index, lines)
        
        inaccuracy_question.append("\n".join(lines[index: next_index]))
        index = next_index
#     for i in inaccuracy_question:
#         print(i)
#         print()
#     print( [extract_and_combine_numbers(topic) for topic in inaccuracy_question])
    # 通过"最长递增子序列"寻找每个精准的题目所在inaccuracy_question对应的下标
    all_question_indexs = longest_increasing_subsequence_index(inaccuracy_question)
    
    # 定义准确的题目列表    
    all_question = []
    # index为all_question_indexs的下标，all_question_indexs[index]为inaccuracy_question的下标
    for index, question_index in enumerate(all_question_indexs):
        if index == len(all_question_indexs) -1 :
            all_question.append("\n".join(inaccuracy_question[question_index:]))
            break
        all_question.append("\n".join(inaccuracy_question[question_index:all_question_indexs[index+1]]))
    
    
    
    if not all_question:
        return None, None
    all_question = find_continuous_sequence(all_question)
    
    # 尝试寻找用于分割答题区与答案区的字符串，返回值为int/str，如果是str则是分割的字符串
    # 本质是在"all_question[-1]"寻找答案关键字等字样
    answer_split_str = find_answer_split_str(all_question)
    
    if isinstance(answer_split_str, str):
        # 如果找到这个拆分的字符串了，则先把最后一道题的内容进行拆分
        all_question[-1] = all_question[-1].split(answer_split_str)[0]
        
    if text.splitlines()[0] in all_question[-1]:
        answer_split_str = text.splitlines()[0]
        # 看看试卷的title是否出现在"all_question[-1]"位置，如果出现则删除
        all_question[-1] = all_question[-1].replace("\n"+text.splitlines()[0]+"\n","")

    return all_question, answer_split_str

# 创建测试样本

In [7]:
#994 题目为 图片+文字（无法用规则提取）
#1087 一 1.2.3 二 1.2.3 （规则还没写）
#1482 解析错误 暂未解决（规则BUG）

In [8]:
import random
random_number = random.randint(0, len(examination_paper_list))
print(random_number)
# test_text = examination_paper_list[random_number]["text"]
test_text = examination_paper_list[65]["text"]

1179


In [9]:
test_text.splitlines()

['2005年高考文科数学![](./notebook/image/media/image1.emf)福建卷![](./notebook/image/media/image1.emf)试题及答案',
 '[源头学子小屋](http://www.xjktyg.com/wxc/) ![](./notebook/image/media/image2.emf)',
 '本试卷分第Ⅰ卷（选择题）和第Ⅱ卷（非选择题）两部分，共150分，考试用时120分钟.考试结束后，将本试卷和答题卡一并交回.',
 '祝各位考生考试顺利！',
 '第I卷（选择题 共60分）',
 '注意事项：',
 '1．答第I卷前，考生务必将自己的姓名、准考证号、考试科目涂写在答题卡上．',
 '2．每小题选出答案后，用铅笔把答题卡上对应题目的答案标号涂黑．如需改动，用橡皮擦干净后，再选涂其它答案标号．不能答在试题卷上．',
 '一、选择题：本大题共12小题，每小题5分，共60分，在每小题给出的四个选项中，只有一项是符合题目要求的.',
 '1．已知集合R\\|，等于 （ ）',
 'A．P B．Q C．{1，2} D．{0，1，2}',
 '2．不等式的解集是 （ ）',
 'A． B．',
 'C． D．',
 '3．已知等差数列中，，则的值是 （ ）',
 'A．15 B．30 C．31 D．64',
 '4．函数在下列哪个区间上是减函数 （ ）',
 'A． B． C． D．',
 '5．下列结论正确的是 （ ）',
 'A．当 B．',
 'C．的最小值为2 D．当无最大值',
 '![](./notebook/image/media/image22.emf)6．函数的图象如图，其中a、b为常数，则下列结论正确的是 （ ）',
 'A．',
 'B．',
 'C．',
 'D．',
 '7．已知直线m、n与平面，给出下列三个命题：',
 '①若',
 '②若',
 '③若',
 '其中真命题的个数是 （ ）',
 'A．0 B．1 C．2 D．3',
 '8．已知的 （ ）',
 'A．充分不必要条件 B．必要不充分条件',
 'C．充要条件 D．既不充分也不必要条件',
 '9．已知定点A、B且\\|AB\\|=4，动点P满足\\|PA\\|－\\|PB\\|=3，则\\|PA\\|的最小值是 

In [10]:
# split_str用于进行下一步的分析（有可能这份试卷是有答题区和答案区，也有可能每道题的答案都在题的下方，还有可能根本没有答案）
all_question, split_str = get_all_question(test_text)
print(f"split_str:\n{split_str}")
if split_str in [-1, 0]:
    print("此试卷无答案")
all_question

split_str:
2005年高考文科数学![](./notebook/image/media/image1.emf)福建卷![](./notebook/image/media/image1.emf)试题及答案


['1．已知集合R\\|，等于 （ ）\nA．P B．Q C．{1，2} D．{0，1，2}',
 '2．不等式的解集是 （ ）\nA． B．\nC． D．',
 '3．已知等差数列中，，则的值是 （ ）\nA．15 B．30 C．31 D．64',
 '4．函数在下列哪个区间上是减函数 （ ）\nA． B． C． D．',
 '5．下列结论正确的是 （ ）\nA．当 B．\nC．的最小值为2 D．当无最大值',
 '![](./notebook/image/media/image22.emf)6．函数的图象如图，其中a、b为常数，则下列结论正确的是 （ ）\nA．\nB．\nC．\nD．',
 '7．已知直线m、n与平面，给出下列三个命题：\n①若\n②若\n③若\n其中真命题的个数是 （ ）\nA．0 B．1 C．2 D．3',
 '8．已知的 （ ）\nA．充分不必要条件 B．必要不充分条件\nC．充要条件 D．既不充分也不必要条件',
 '9．已知定点A、B且\\|AB\\|=4，动点P满足\\|PA\\|－\\|PB\\|=3，则\\|PA\\|的最小值是 （ ）\nA． B． C． D．5',
 '10．从6人中选4人分别到巴黎、伦敦、悉尼、莫斯科四个城市游览，要求每个城市有一人游览，每人只游览一个城市，且这6人中甲、乙两人不去巴黎游览，则不同的选择方案共有 （ ）\nA．300种 B．240种 C．144种 D．96种',
 '11．如图，长方体ABCD---A~1~B~1~C~1~D~1~中，AA~1~=AB=2，\n![](./notebook/image/media/image36.emf)AD=1，点E、F、G分别是DD~1~、AB、CC~1~的中\n点，则异面直线A~1~E与GF所成的角是（ ）\nA． B．\nC． D．',
 '12．是定义在R上的以3为周期的偶函数，且，则方程=0在区间（0，6）内解的个数的最小值是 （ ）\nA．5 B．4 C．3 D．2\n第Ⅱ卷（非选择题 共90分）\n二、填空题：本大题共4小题，每小题4分，共16分，把答案填在答题卡的相应位置![](./notebook/image/media/image2.emf)',
 '13．展开式中的常数项是 [ ]{.underline} （用数字作答）![](./no

In [116]:
if split_str == 1:
    for row in align_answers_in_questions(all_question):
        print()
        print(row["question"])
        print("-------------------------------------------------------")
        print(f"{row['answer']}")
        print()
        print("============================================================")

# 寻找答案（试卷存在答题区与答案区）

In [10]:
def get_all_answer_sequence(answer_area_string):
    lines = answer_area_string.splitlines()
    
    inaccuracy_answers = []

    index = find_next_question_index(0, lines)
    while index < len(lines):
#         print(lines[index])
#         print()
        next_index = find_next_question_index(index, lines)
       
        inaccuracy_answers.append("\n".join(lines[index: next_index]))
        index = next_index
#     print("============")
#     for i in inaccuracy_answers:
#         print(i)
#         print()
        
#     print([extract_and_combine_numbers(topic) for topic in inaccuracy_answers])
    inaccuracy_answer_indexes = longest_increasing_subsequence_index(inaccuracy_answers)
   
    processed_inaccuracy_answers = []
    for index, answer_index in enumerate(inaccuracy_answer_indexes):
        if index == len(inaccuracy_answer_indexes) -1 :
            processed_inaccuracy_answers.append(inaccuracy_answers[answer_index])
            break
        processed_inaccuracy_answers.append("\n".join(inaccuracy_answers[answer_index:inaccuracy_answer_indexes[index+1]]))
        
    return refine_answers(processed_inaccuracy_answers)[::-1]

In [11]:
answer_str = generate_answer_area_string(test_text, split_str)

In [12]:
answer_str.splitlines()

['',
 '参考答案与试题解析',
 '一．选择题：在每小题给出的四个选项中，只有一项是符合题目要求的.共8小题,每小题5分,共40分.',
 '1．（5分）已知集合A={x∈R\\|\\|x\\|≤2}，B={x∈R\\|x≤1}，则A∩B=（）',
 'A．（﹣∞，2\\] B．\\[1，2\\] C．\\[﹣2，2\\] D．\\[﹣2，1\\]',
 '【分析】先化简集合A，解绝对值不等式可求出集合A，然后根据交集的定义求出A∩B即可．',
 '【解答】解：∵A={x\\|\\|x\\|≤2}={x\\|﹣2≤x≤2}',
 '∴A∩B={x\\|﹣2≤x≤2}∩{x\\|x≤1，x∈R}={x\\|﹣2≤x≤1}',
 '故选：D．',
 '【点评】本题主要考查了绝对值不等式，以及交集及其运算，同时考查了运算求解的能力，属于基础题．',
 '2．（5分）设变量x，y满足约束条件![](./notebook/image/media/image27.png)，则目标函数z=y﹣2x的最小值为（）',
 'A．﹣7 B．﹣4 C．1 D．2',
 '【分析】先根据条件画出可行域，设z=y﹣2x，再利用几何意义求最值，将最小值转化为y轴上的截距最小，只需求出直线z=y﹣2x，过可行域内的点B（5，3）时的最小值，从而得到z最小值即可．',
 '【解答】解：设变量x、y满足约束条件 ![](./notebook/image/media/image28.png)，',
 '在坐标系中画出可行域三角形，',
 '平移直线y﹣2x=0经过点A（5，3）时，y﹣2x最小，最小值为：﹣7，',
 '则目标函数z=y﹣2x的最小值为﹣7．',
 '故选：A．',
 '![](./notebook/image/media/image29.png)',
 '【点评】借助于平面区域特性，用几何方法处理代数问题，体现了数形结合思想、化归思想．线性规划中的最优解，通常是利用平移直线法确定．',
 '3．（5分）阅读如图所示的程序框图，运行相应的程序，则输出n的值为（）',
 '![](./notebook/image/media/image30.png)',
 'A．7 B．6 C．5 D．4',
 '【分析】利用循环结构可知道需要循环4次方可得到S←2，因此输出的n←4．',
 '【

In [13]:
for i in get_all_answer_sequence(answer_area_str_process(answer_str)):
    print(i)
    print()

1．（5分）已知集合A={x∈R\|\|x\|≤2}，B={x∈R\|x≤1}，则A∩B=（）
A．（﹣∞，2\] B．\[1，2\] C．\[﹣2，

2．（5分）设变量x，y满足约束条件![](./notebook/image/media/image27.png)，则目标函数z=y﹣2x的最小值为（）
A．﹣7 B．﹣4 C．1 D．2
【分析】先根据条件画出可行域，设z=y﹣2x，再利用几何意义求最值，将最小值转化为y轴上的截距最小，只需求出直线z=y﹣2x，过可行域内的点B（5，3）时的最小值，从而得到z最小值即可．
【解答】解：设变量x、y满足约束条件 ![](./notebook/image/media/image28.png)，
在坐标系中画出可行域三角形，
平移直线y﹣2x=0经过点A（5，3）时，y﹣2x最小，最小值为：﹣7，
则目标函数z=y﹣2x的最小值为﹣7．
故选：A．
![](./notebook/image/media/image29.png)
【点评】借助于平面区域特性，用几何方法处理代数问题，体现了数形结合思想、化归思想．线性规划中的最优解，通常是利用平移直线法确定．

3．（5分）阅读如图所示的程序框图，运行相应的程序，则输出n的值为（）
![](./notebook/image/media/image30.png)
A．7 B．6 C．5 D．4
【分析】利用循环结构可知道需要循环4次方可得到S←2，因此输出的n←4．
【解答】解：由程序框图可知：S=2=0+（﹣1）^1^×1+（﹣1）^2^×2+（﹣1）^3^×3+（﹣1）^4^×4，
因此当n=4时，S←2，满足判断框的条件，故跳出循环程序．
故输出的n的值为

4．（5分）设a，b∈R，则"（a﹣b）a^2^＜0"是"a＜b"的（）
A．充分而不必要条件 B．必要而不充分条件
C．充要条件 D．既不充分也不必要条件
【分析】根据充分必要条件定义判断，结合不等式求解．
【解答】解：∵a，b∈R，则（a﹣b）a^2^＜0，
∴a＜b成立，
由a＜b，则a﹣b＜0，"（a﹣b）a^2^≤0，
所以根据充分必要条件的定义可的判断：
a，b∈R，则"（a﹣b）a^2^＜0"是a＜b的充分不必要条件，
故选：A．
【点评】本题考查了不等式，充分必要条件的定义，属于容易题．

5．（5

# 对齐

In [14]:
def alignment_answer(all_question, answer_str):
    questions_with_answer = []
    all_answer = get_all_answer_sequence(answer_area_str_process(answer_str))

    questions_map = {extract_and_combine_numbers(question):question for question in reversed(all_question)} 
    answer_map = {extract_and_combine_numbers(answer):answer for answer in reversed(all_answer)}

    for sequence_number in questions_map:
        questions_with_answer.append({"question":questions_map.get(sequence_number),"answer":answer_map.get(sequence_number, None)})
    return questions_with_answer

In [15]:
def check_sequence_is_full(questions_with_answer):
    for index, question_map in enumerate(questions_with_answer):
        if index == len(questions_with_answer)-1:
            break
        if extract_and_combine_numbers(question_map["question"]) != extract_and_combine_numbers(questions_with_answer[index+1]["question"])+1:
            return False
    return True

In [16]:
for row in alignment_answer(all_question, answer_str):
    print()
    print(row["question"])
    print("-------------------------------------------------------")
    print(f"{row['answer']}")
    print()
    print("============================================================")


20．（14分）设a∈\[﹣2，0\]，已知函数![](./notebook/image/media/image25.png)
（Ⅰ） 证明f（x）在区间（﹣1，1）内单调递减，在区间（1，+∞）内单调递增；
（Ⅱ） 设曲线y=f（x）在点P~i~（x~i~，f（x~i~））（i=1，2，3）处的切线相互平行，且x~1~x~2~x~3~≠0，证明![](./notebook/image/media/image26.png)．
-------------------------------------------------------
20．（14分）设a∈\[﹣2，0\]，已知函数![](./notebook/image/media/image165.png)
（Ⅰ） 证明f（x）在区间（﹣1，1）内单调递减，在区间（1，+∞）内单调递增；
（Ⅱ） 设曲线y=f（x）在点P~i~（x~i~，f（x~i~））（i=1，2，3）处的切线相互平行，且x~1~x~2~x~3~≠0，证明![](./notebook/image/media/image166.png)．
【分析】（Ⅰ）令![](./notebook/image/media/image167.png)，![](./notebook/image/media/image168.png)．分别求导即可得到其单调性；
（Ⅱ）由（Ⅰ）可知：f′（x）在区间（﹣∞，0）内单调递减，在区间![](./notebook/image/media/image169.png)内单调递减，在区间![](./notebook/image/media/image170.png)内单调递增．
已知曲线y=f（x）在点P~i~（x~i~，f（x~i~））（i=1，2，3）处的切线相互平行，可知x~1~，x~2~，x~3~互不相等，利用导数的几何意义可得![](./notebook/image/media/image171.png)．
不妨x~1~＜0＜x~2~＜x~3~，根据以上等式可得![](./notebook/image/media/image172.png)，从而![](./notebook/image/media/image173.png)．设g（x）=3x^2^﹣（a+3）x+a，利用二次函数的单调性可得![](./notebo

# 生产测试

In [30]:
import csv
from tqdm import tqdm

csv_path = "./questions_with_answers.csv"
loss_file_path_set = set()
with open(csv_path, 'a', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['file_path', 'question', 'answer', 'is_full', 'is_error']  # 列名
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for examination_paper in tqdm(examination_paper_list):

        file_path = examination_paper["file_path"]
        text = examination_paper["text"]
        
        all_question, split_str = get_all_question(text)
        
        # 判断试卷你是否有答案 -1 识别不到 0 无答案 
        if split_str in [-1, 0]: 
            loss_file_path_set.add(file_path)
            continue
        # 无法找到题目
        if not all_question:
            loss_file_path_set.add(file_path)
            continue
        
        # 答案在每道题的下方split_str == 1，如果答案在卷子下方与题目隔离则split_str返回隔离答案和题目那一行的str字符串
        if split_str == 1:
            questions_with_answers = align_answers_in_questions(all_question)
        else:
            answer_str = generate_answer_area_string(text, split_str)
            questions_with_answers = alignment_answer(all_question, answer_str)
            
        if questions_with_answers is []:
            loss_file_path_set.add(file_path)
            continue
            
        try:
            is_full = check_sequence_is_full(questions_with_answers)
        except:
            loss_file_path_set.add(file_path)
            continue
            
        for question_with_answer in questions_with_answers:
            
            if question_with_answer is None:
                question_with_answer = {"question":None,"answer":None,"is_full":is_full,"file_path":file_path,"is_error":True}
            question_with_answer.update({"is_full":is_full,"file_path":file_path,"is_error":False})
            if len(question_with_answer) == 5:
                writer.writerow(question_with_answer)
        


100%|██████████████████████████████████████████████████████████████████████████████| 1615/1615 [00:44<00:00, 35.94it/s]


In [31]:
len(loss_file_path_set)/len(examination_paper_list)

0.1931888544891641

# 检验

In [37]:
import pandas as pd

df = pd.read_csv(csv_path)
df_include_answer = df[df['answer'].notnull()]

count_file_path = df_include_answer['file_path'].nunique()
count_file_path/len(examination_paper_list)

0.7473684210526316

In [55]:
len(df_include_answer)

20462

In [56]:
for _, row in df_include_answer.iloc[20000:20050].iterrows():
    print(row['question'])
    print("************************************************************")
    print(row['answer'])
    print("\n=======================================================================\n")


19.在中，，为BC边上的高，，则BC的长为\_\_\_\_\_\_\_\_\_\_\_．
************************************************************
【答案】7或5
【解析】
【分析】
如图所示，分D在BC之间和BC延长线上两种情况考虑，先由求出BD，再求出BC的长．
【详解】解：如图，∵在Rt△ABD中，，，
![](./notebook/image/media/image115.png)
∴，即：，
∴，
当D在BC之间时，BC=BD+CD=6+1=7；
当D在BC延长线上时，BC=BD-CD=6-1=5；
故答案为：7或5．
【点睛】此题主要考查了解三角形，根据已知得出两种符合要求![](./notebook/image/media/image100.wmf)图形，即三角形为钝角三角形或锐角三角形分别分析是解题关键．


20.如图，在菱形中，对角线相交于点O，点E在线段BO上，连接AE，若，，,则线段AE的长为\_\_\_\_\_．
![](./notebook/image/media/image124.png)
************************************************************
【答案】【解析】
【分析】
设BE=x，根据菱形性质可得到AB= AD=CD=2x，进而得到，解得x值，根据勾股定理即可求得AE值．
【详解】解：设BE=x，
∵菱形，
∴AB= AD=CD=2x，
∵，
∴,
∴BD=3x，
∴OB=OD=，
∴，
∴x=2，
∴AB=4，BE=2，
∴，
∴,
故答案为：．
【点睛】本题考查菱形的性质结合勾股定理的应用，熟练掌握菱形性质是解题的关键．
三、解答题


21.先化简，再求代数式的值，其中
************************************************************
【答案】原式，
【解析】
【分析】
先根据分式的运算法则化简，再利用求得x的值，代入计算即可．
【详解】解：原式
，
∵，
∴
，
∴原式
．
【点睛】本题考查了分式的化简求值，特殊角的三角函数值，二次根式的计算，熟练掌握相关运算法则是解决本题的关键．


22.如图，方格纸中每个小正方形的边长为1，线段AB和